In [1]:
# Kütüphaneleri yüklüyoruz
import pandas as pd
import re
import numpy as np
import gensim

In [2]:
data=pd.read_csv('BigAll.csv') # Hürriyetten scrape edilmiş haberler
data.drop(columns=['Unnamed: 0'],inplace=True)
data.columns = ['Haber','Ozet','Baslık','Tarıh'] # Verimizin sütun isimlerini değiştiriyoruz

In [3]:
# Verimiz çok büyük olduğu için 7000 lik kısmıyla çalışıcaz.
# Dilerseniz daha büyük veriyle çalışabilirsiniz.
data2=data.iloc[0:7000]

In [4]:
data2.head(8)

,Haber,Ozet,Baslık,Tarıh
0,"OCI'dan yapılan açıklamada, OCI Çimento A.Ş'ni...","Orascom Construction Industries (OCI), Batı An...",OCI Batı Çimento'da çağrı için onay aldı,26.03.2007
1,\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\nDün düzenlen...,Kayseri'de dün Dışişleri Bakanı ve Başbakan Ya...,DSİ'den 147 milyon ytl'lik yatırım,26.03.2007
2,"\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\nOlay, Reyhan...",Hatay’ın Reyhanlı İlçesi’ndeki Afrin Çayı kena...,Afrin Çayı’nda boğuldu,26.03.2007
3,\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\nSarıgöl İlçe...,Manisa'nın Sarıgöl ilçesindeki üzüm üreticiler...,Üzüm üreticisi “idareli su” için damlama yönte...,26.03.2007
4,\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\nMusul polisi...,"Irak genelinde şiddet olayları sürerken, bugün...","Irak'ta 6 kişi öldü, 2 ceset bulundu",26.03.2007
5,\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\nOlay önceki ...,"BOLU'da, zihinsel engelli 21 yaşındaki Y.T.'ye...",Zihinsel engelli genç kıza ormanda tecavüz idd...,26.03.2007
6,\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\n BAKAN UNAKI...,"Maliye Bakanı Kemal Unakıtan'a mısır atıldı, S...",Unakıtan'a mısır attılar,26.03.2007
7,"\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\nÇalışkan, fa...","AK Parti Adana Milletvekili Abdullah Çalışkan,...",Adana Tekel Sigara Fabrikası üretime başlıyor,26.03.2007


In [ ]:
data['Cleaned'] = pd.Series() # Haberlerimizi temizledikten sonra kaydetmek için değişken atıyoruz

In [ ]:
# Haberlerin içerdiği gereksiz karakterleri temizlemek için fonksiyon

def cleantext(text2):
    text = re.sub(r"\r", "", text2)
    text = re.sub(r"\n", "", text)
    text = re.sub(r"/n", "", text)
    text = re.sub(r"/", "", text)
    text = re.sub(r"\\", "", text)
    text = re.sub('�', '',text)
    text = re.sub('/', '',text)
    text = re.sub("\'", '',text)
    return text

In [ ]:
data2['Cleaned'] = data2['Haber'].apply(cleantext) # Haberlerimizi temizleyip 'Cleaned' sütununda topluyoruz

In [ ]:
data2.to_csv('7000_haber.csv') # Görsel modelde kullanılması için 7000 haberi kaydediyoruz.

In [ ]:
# Haber benzerliği için class'ımız

class DocSim(object):
    def __init__(self, w2v_model , stopwords=[]):
        self.w2v_model = w2v_model
        self.stopwords = stopwords

    def vectorize(self, doc):
        """Identify the vector values for each word in the given document"""
        doc = doc.lower()
        words = [w for w in doc.split(" ") if w not in self.stopwords]
        word_vecs = []
        for word in words:
            try:
                vec = self.w2v_model[word]
                word_vecs.append(vec)
            except KeyError:
                # Kelime sözlükte yoksa yok sayıyoruz.
                pass

        # Döküman vektörleri tüm kelime vektörlerinin ortalaması olduğunu kabul ediyoruz.
        # Not: Daha iyi yollar da var
        vector = np.mean(word_vecs, axis=0)
        return vector


    def _cosine_sim(self, vecA, vecB):
        """İki vektör arasındaki cosinüs benzerliğini hesaplıyoruz"""
        csim = np.dot(vecA, vecB) / (np.linalg.norm(vecA) * np.linalg.norm(vecB))
        if np.isnan(np.sum(csim)):
            return 0
        return csim

    def calculate_similarity(self, source_doc, target_docs=[], threshold=0):
        """Verilen haberle diğer haberleri karşılaştırarak benzerlik skorlarını hesaplıyıp
            sonuç olarak döndürüyoruz."""
        if isinstance(target_docs, str):
            target_docs = [target_docs]

        source_vec = self.vectorize(source_doc)
        results = []
        for doc in target_docs:
            target_vec = self.vectorize(doc)
            sim_score = self._cosine_sim(source_vec, target_vec)
            if sim_score > threshold:
                results.append({
                    'score' : sim_score,
                    'doc' : doc
                })
            results.sort(key=lambda k : k['score'] , reverse=True)

        return results

In [ ]:
from gensim.models.keyedvectors import KeyedVectors

# Türkçe kelimelerle önceden eğitilmiş Word2Vec modelimizi kullanıyoruz.
model_path = 'trmodel'
w2v_model = KeyedVectors.load_word2vec_format(model_path, binary=True)

In [ ]:
# Türkçe stopwordsları text dosyasından okuyoruz.

file=open('KaVe2019/stopwords-tr.txt', 'r')
stopwords = list(file.read().split())

In [ ]:
ds = DocSim(w2v_model,stopwords)

In [ ]:
source_doc=data2['Cleaned'][912] #Karşılaştırmak istediğimiz haber
target_docs=list(data2['Cleaned'].iloc[1:7000]) #Karşılaştırıldığı diğer haberler

sim_scores = ds.calculate_similarity(source_doc, target_docs) #Benzerliğin hesaplanması

In [ ]:
source_doc # Bizim haberimiz

In [ ]:
sim_scores[1:11] # En yakın 10 haberi skorları ve içerikleriyle beraber döndürüyoruz.